In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import pathlib
import numpy as np
import pandas as pd
import sklearn.metrics as metrics
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
pixel = 128
batch_size = 128
data_root_orig = './testData'
data_root = pathlib.Path(data_root_orig)
print(data_root)
label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
#label_names.remove('.ipynb_checkpoints')
print(label_names)
label_to_index = dict((name, i) for i, name in enumerate(label_names))

testData
['GabyNg', 'LeeSeng', 'XiaoYan']


In [3]:
test_set_df = pd.read_csv('./test_GabyNg_set.csv')
print(len(test_set_df))

21


In [4]:
rescale_datagen = ImageDataGenerator(
    rescale=1.0/255,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rotation_range=20,
    zoom_range=0.10,
    shear_range=0.15,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest')
test_rescale_generator = rescale_datagen.flow_from_dataframe(dataframe=test_set_df, x_col="filename", y_col="label",
                                                  class_mode="categorical", target_size=(pixel, pixel),
                                                  shuffle=False,
                                                  batch_size=batch_size)

Found 21 validated image filenames belonging to 3 classes.


In [5]:
modelname = 'model/face_cnn_rescale_b128_e100'
filename = modelname+'.h5'
model = load_model(filename)
pred = model.predict(test_rescale_generator)

predout = np.argmax(pred, axis=1)
testout = test_set_df['label'].apply(lambda x: label_to_index[x]).values
print('Test Results for CNN using face only images in rescale data generator')
print('testout:', testout)
print('predout:', predout)

testScores = metrics.accuracy_score(testout, predout)
confusion = metrics.confusion_matrix(testout, predout)

print("Batch size is "+ str(batch_size))
print("Best accuracy (on testing dataset): %.2f%%" % (testScores * 100))
print(metrics.classification_report(testout, predout, labels=[0,1,2],target_names=label_names, digits=4))
print(confusion)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Test Results for CNN using face only images in rescale data generator
testout: [0 0 0 0 0 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1]
predout: [0 0 0 0 0 2 0 2 0 1 1 1 2 0 0 0 1 2 0 0 1]
Batch size is 128
Best accuracy (on testing dataset): 47.62%
              precision    recall  f1-score   support

      GabyNg     0.4167    1.0000    0.5882         5
     LeeSeng     0.4000    0.2857    0.3333         7
     XiaoYan     0.7500    0.3333    0.4615         9

    accuracy                         0.4762        21
   macro avg     0.5222    0.5397    0.4610        21
weighted avg     0.5540    0.4762    0.4490        21

[[5 0 0]
 [4 2 1]
 [3 3 3]]


In [6]:
modelname = 'model/face_resnet_rescale_b128_e100'
filename = modelname+'.h5'
model = load_model(filename)
pred = model.predict(test_rescale_generator)

predout = np.argmax(pred, axis=1)
testout = test_set_df['label'].apply(lambda x: label_to_index[x]).values
print('Test Results for ResNet using face only images in rescale data generator')
print('testout:', testout)
print('predout:', predout)

testScores = metrics.accuracy_score(testout, predout)
confusion = metrics.confusion_matrix(testout, predout)

print("Batch size is "+ str(batch_size))
print("Best accuracy (on testing dataset): %.2f%%" % (testScores * 100))
print(metrics.classification_report(testout, predout, labels=[0,1,2],target_names=label_names, digits=4))
print(confusion)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Test Results for ResNet using face only images in rescale data generator
testout: [0 0 0 0 0 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1]
predout: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1]
Batch size is 128
Best accuracy (on testing dataset): 23.81%
              precision    recall  f1-score   support

      GabyNg     0.0000    0.0000    0.0000         5
     LeeSeng     0.2632    0.7143    0.3846         7
     XiaoYan     0.0000    0.0000    0.0000         9

    accuracy                         0.2381        21
   macro avg     0.0877    0.2381    0.1282        21
weighted avg     0.0877    0.2381    0.1282        21

[[0 5 0]
 [2 5 0]
 [0 9 0]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [7]:
sample_datagen = ImageDataGenerator(
    samplewise_center=True,
    samplewise_std_normalization=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rotation_range=20,
    zoom_range=0.10,
    shear_range=0.15,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest')
test_sample_generator = sample_datagen.flow_from_dataframe(dataframe=test_set_df, x_col="filename", y_col="label",
                                                  class_mode="categorical", target_size=(pixel, pixel),
                                                  shuffle=False,
                                                  batch_size=batch_size)

Found 21 validated image filenames belonging to 3 classes.


In [8]:
modelname = 'model/face_cnn_sample_b128_e100'
filename = modelname+'.h5'
model = load_model(filename)
pred = model.predict(test_sample_generator)

predout = np.argmax(pred, axis=1)
testout = test_set_df['label'].apply(lambda x: label_to_index[x]).values
print('Test Results for CNN using face only images in samplewise data generator')
print('testout:', testout)
print('predout:', predout)

testScores = metrics.accuracy_score(testout, predout)
confusion = metrics.confusion_matrix(testout, predout)

print("Batch size is "+ str(batch_size))
print("Best accuracy (on testing dataset): %.2f%%" % (testScores * 100))
print(metrics.classification_report(testout, predout, labels=[0,1,2],target_names=label_names, digits=4))
print(confusion)

Test Results for CNN using face only images in samplewise data generator
testout: [0 0 0 0 0 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1]
predout: [0 0 0 0 2 1 0 0 0 0 0 2 0 0 0 0 0 0 0 0 1]
Batch size is 128
Best accuracy (on testing dataset): 28.57%
              precision    recall  f1-score   support

      GabyNg     0.2353    0.8000    0.3636         5
     LeeSeng     0.5000    0.1429    0.2222         7
     XiaoYan     0.5000    0.1111    0.1818         9

    accuracy                         0.2857        21
   macro avg     0.4118    0.3513    0.2559        21
weighted avg     0.4370    0.2857    0.2386        21

[[4 0 1]
 [6 1 0]
 [7 1 1]]


In [9]:
modelname = 'model/face_resnet_sample_b128_e100'
filename = modelname+'.h5'
model = load_model(filename)
pred = model.predict(test_sample_generator)

predout = np.argmax(pred, axis=1)
testout = test_set_df['label'].apply(lambda x: label_to_index[x]).values
print('Test Results for ResNet using face only images in samplewise data generator')
print('testout:', testout)
print('predout:', predout)

testScores = metrics.accuracy_score(testout, predout)
confusion = metrics.confusion_matrix(testout, predout)

print("Batch size is "+ str(batch_size))
print("Best accuracy (on testing dataset): %.2f%%" % (testScores * 100))
print(metrics.classification_report(testout, predout, labels=[0,1,2],target_names=label_names, digits=4))
print(confusion)

Test Results for ResNet using face only images in samplewise data generator
testout: [0 0 0 0 0 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1]
predout: [1 0 0 0 0 1 0 0 0 0 1 0 1 0 1 0 1 1 1 0 1]
Batch size is 128
Best accuracy (on testing dataset): 42.86%
              precision    recall  f1-score   support

      GabyNg     0.3333    0.8000    0.4706         5
     LeeSeng     0.5556    0.7143    0.6250         7
     XiaoYan     0.0000    0.0000    0.0000         9

    accuracy                         0.4286        21
   macro avg     0.2963    0.5048    0.3652        21
weighted avg     0.2646    0.4286    0.3204        21

[[4 1 0]
 [2 5 0]
 [6 3 0]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [10]:
sample_rescale_datagen = ImageDataGenerator(
    samplewise_center=True,
    samplewise_std_normalization=True,
    rescale=1.0/255,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rotation_range=20,
    zoom_range=0.10,
    shear_range=0.15,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest')
test_sample_rescale_generator = sample_rescale_datagen.flow_from_dataframe(dataframe=test_set_df, x_col="filename", y_col="label",
                                                  class_mode="categorical", target_size=(pixel, pixel),
                                                  shuffle=False,
                                                  batch_size=batch_size)

Found 21 validated image filenames belonging to 3 classes.


In [11]:
modelname = 'model/face_cnn_sample_rescale_b128_e100'
filename = modelname+'.h5'
model = load_model(filename)
pred = model.predict(test_sample_rescale_generator)

predout = np.argmax(pred, axis=1)
testout = test_set_df['label'].apply(lambda x: label_to_index[x]).values
print('Test Results for CNN using face only images in samplewise and rescale data generator')
print('testout:', testout)
print('predout:', predout)

testScores = metrics.accuracy_score(testout, predout)
confusion = metrics.confusion_matrix(testout, predout)

print("Batch size is "+ str(batch_size))
print("Best accuracy (on testing dataset): %.2f%%" % (testScores * 100))
print(metrics.classification_report(testout, predout, labels=[0,1,2],target_names=label_names, digits=4))
print(confusion)

Test Results for CNN using face only images in samplewise and rescale data generator
testout: [0 0 0 0 0 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1]
predout: [0 0 0 0 0 0 0 0 0 1 2 1 1 0 0 0 0 0 0 0 1]
Batch size is 128
Best accuracy (on testing dataset): 33.33%
              precision    recall  f1-score   support

      GabyNg     0.3125    1.0000    0.4762         5
     LeeSeng     0.2500    0.1429    0.1818         7
     XiaoYan     1.0000    0.1111    0.2000         9

    accuracy                         0.3333        21
   macro avg     0.5208    0.4180    0.2860        21
weighted avg     0.5863    0.3333    0.2597        21

[[5 0 0]
 [6 1 0]
 [5 3 1]]


In [12]:
modelname = 'model/face_resnet_sample_rescale_b128_e100'
filename = modelname+'.h5'
model = load_model(filename)
pred = model.predict(test_sample_rescale_generator)

predout = np.argmax(pred, axis=1)
testout = test_set_df['label'].apply(lambda x: label_to_index[x]).values
print('Test Results for ResNet using face only images in samplewise and rescale data generator')
print('testout:', testout)
print('predout:', predout)

testScores = metrics.accuracy_score(testout, predout)
confusion = metrics.confusion_matrix(testout, predout)

print("Batch size is "+ str(batch_size))
print("Best accuracy (on testing dataset): %.2f%%" % (testScores * 100))
print(metrics.classification_report(testout, predout, labels=[0,1,2],target_names=label_names, digits=4))
print(confusion)

Test Results for ResNet using face only images in samplewise and rescale data generator
testout: [0 0 0 0 0 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1]
predout: [0 0 1 1 0 2 2 2 1 0 2 0 0 0 1 1 1 1 1 0 1]
Batch size is 128
Best accuracy (on testing dataset): 61.90%
              precision    recall  f1-score   support

      GabyNg     0.3750    0.6000    0.4615         5
     LeeSeng     0.6667    0.8571    0.7500         7
     XiaoYan     1.0000    0.4444    0.6154         9

    accuracy                         0.6190        21
   macro avg     0.6806    0.6339    0.6090        21
weighted avg     0.7401    0.6190    0.6236        21

[[3 2 0]
 [1 6 0]
 [4 1 4]]


In [13]:
modelname = 'model/face_resnet2_sample_rescale_b128_e100'
filename = modelname+'.h5'
model = load_model(filename)
pred = model.predict(test_sample_rescale_generator)

predout = np.argmax(pred, axis=1)
testout = test_set_df['label'].apply(lambda x: label_to_index[x]).values
print('Test Results for ResNet V2 using face only images in samplewise and rescale data generator')
print('testout:', testout)
print('predout:', predout)

testScores = metrics.accuracy_score(testout, predout)
confusion = metrics.confusion_matrix(testout, predout)

print("Batch size is "+ str(batch_size))
print("Best accuracy (on testing dataset): %.2f%%" % (testScores * 100))
print(metrics.classification_report(testout, predout, labels=[0,1,2],target_names=label_names, digits=4))
print(confusion)

Test Results for ResNet V2 using face only images in samplewise and rescale data generator
testout: [0 0 0 0 0 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1]
predout: [0 0 0 0 0 2 0 1 1 0 1 0 1 0 1 0 1 1 1 0 1]
Batch size is 128
Best accuracy (on testing dataset): 52.38%
              precision    recall  f1-score   support

      GabyNg     0.4545    1.0000    0.6250         5
     LeeSeng     0.5556    0.7143    0.6250         7
     XiaoYan     1.0000    0.1111    0.2000         9

    accuracy                         0.5238        21
   macro avg     0.6700    0.6085    0.4833        21
weighted avg     0.7220    0.5238    0.4429        21

[[5 0 0]
 [2 5 0]
 [4 4 1]]
